In [1]:
# imports
%load_ext autoreload
%autoreload 2
import warnings
import os
from etils import ecolab
import numpy as np
from asammdf import MDF
from pathlib import Path
import logging
from collections import  defaultdict
import pandas as pd
import datetime
import matplotlib.pyplot as plt

import utils.can_log_utils as utls
from utils.dbc_db import DbcDb


logging.getLogger('asammdf').setLevel(logging.DEBUG)
%connect_info

ldf is not supported
xls is not supported
xlsx is not supported


{"key":"76db4b78-9a8c-48cc-a091-e9d6649de9e0","signature_scheme":"hmac-sha256","transport":"tcp","ip":"127.0.0.1","hb_port":9005,"control_port":9006,"shell_port":9007,"stdin_port":9008,"iopub_port":9009,"kernel_name":"python3127jvsc74a57bd0e7c25480a53691149789381bdda99fb7f8d344b19173f1535ebe8bfb62cf4ee5"}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing c:\Users\meleshin\AppData\Roaming\jupyter\runtime\kernel-v3b76352b3902238f1f249ceee0e00c4309be8ed25.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
# set input data
dbc_dir=r'C:\d\data\OneDrive - Arrival\gdrive\work\VAN\DBC\ucs_dbc-0.12.0.0'
# log_file=r'C:\D\downloads\VI-15803\R12_D19_2024-10-31_10-48-44.mf4'
# log_file=r'C:\D\work\py_work\UCSDT_utils\can_log_mdf_parse\input\r11_D19\R11_canlog_50640_20240926_142715.mf4'
log_file=r'C:\D\work\py_work\UCSDT_utils\can_log_mdf_parse\input\r12_D19\canlog_51940_20241107_122852.mf4'
log_file=r'C:\D\downloads\VI-15803\vcd_12_drop2_27_09_24.mf4'
# file_export='C:\D\downloads\VI-15803\R12_D19_2024-10-31_10-48-44_1.xlsx'
file_export=utls.get_report_filename(log_file)
CAN_MAPS_FILE=r'input\VAN_CAN_MAPS.xlsx'
CAN_MAPS_SHEET_ECU='ECU_NAMES'
f_time=[0,100] # selection of the log

In [3]:
utls.prepare_report_file(file_export,dbc_dir,log_file,f_time)

dropped file and written info page into C:\D\downloads\VI-15803\vcd_12_drop2_27_09_24.xlsx


In [4]:
# load all data from can log
mdf = MDF(log_file)
if f_time:
    mdf=mdf.cut(f_time[0],f_time[1],whence=1,time_from_zero=1)

# load wihtout payload
mdf_d = utls.mdf_get_can_msgs(mdf,get_timestat=True,get_time=True)
# if no need to analyze dt vs time or distribution of specific messages - get_time=False

data collected into groups by (CAN,MSG_ID,IsExtended)
number of records: 287


In [5]:
# load dbc databases
dbc_db=DbcDb(dbc_dir,False)

total number of id: 1715
total number of dbc.msg: 3143
number of dublicated id: 1245
number of dublicated pgn: 1210


In [6]:
# load ECU names
ECU_NAME_MAP=pd.read_excel(CAN_MAPS_FILE,sheet_name=CAN_MAPS_SHEET_ECU,index_col='node_id_dec')
ECU_NAME_MAP.head();
ECU_NAME_MAP['node_name'][90];


,node_name,node_id_hex
node_id_dec,,
3,IDU,3
17,ADAS,11
23,HMI,17
25,FBIO3,19
59,AVAS,3B


'DCU'

In [7]:
# process prepare tables
(mdf_d,mdf_df)=utls.proc_mdf_all(mdf,mdf_d,dbc_db,ecu_name_map=ECU_NAME_MAP)
mdf_df.attrs;
mdf_df.head()

{'mean': 10, 'cv': 10}


{'mdf.file_history': '[FHBLOCK(time=2024-11-19 20:35:18.921061+04:00, comment=<FHcomment>\n    <TX>Cut from start of measurement to 100.0s</TX>\n    <tool_id>asammdf</tool_id>\n    <tool_vendor>asammdf</tool_vendor>\n    <tool_version>8.0.1</tool_version>\n</FHcomment>)]',
 'mdf.start_time': datetime.datetime(2024, 9, 27, 9, 24, 5, 37000),
 'mdf.name': WindowsPath('__new__.mf4')}

CAN_Bus    msg_id  is_extended  msg_pgn msg_pdu  \
CAN_Bus msg_id   is_extended                                                    
1       1832     0                  1      1832            0      NaN    None   
        15739738 1                  1  15739738            1  61483.0    PDU2   
        19422042 1                  1  19422042            1  75776.0    PDU1   
        19553114 1                  1  19553114            1  76288.0    PDU1   
        22174282 1                  1  22174282            1  86528.0    PDU1   

                              msg_sa  msg_da   dt_mean  dt_median    dt_min  \
CAN_Bus msg_id   is_extended                                                  
1       1832     0               NaN     NaN  0.099997   0.099996  0.099797   
        15739738 1              90.0     NaN  0.110046   0.100001  0.099820   
        19422042 1              90.0    91.0  0.022282   0.020001  0.019801   
        19553114 1              90.0    91.0  1.000041   1.000042  0.999907   
        22174282 1              74.0    90.0  0.007121   0.003214  0.000118   

                              ...        dt_cv  dt_nobs  \
CAN_Bus msg_id   is_extended  ...                         
1       1832     0            ...     0.061978    999.0   
        15739738 1            ...   274.814440    908.0   
        19422042 1            ...   685.839803   4487.0   
        19553114 1            ...     0.003443     89.0   
        22174282 1            ...  2388.976149   6406.0   

                                                                            dbc  \
CAN_Bus msg_id   is_extended                                                      
1       1832     0            [{'dbc': 'Arrival_OBC_v1.dbc', 'msg': 'OBC_ACT...   
        15739738 1            [{'dbc': 'ADAS_J1939.dbc', 'msg': 'CN', 'is_j1...   
        19422042 1            [{'dbc': 'Arrival_MBMS.dbc', 'msg': 'BMS_Power...   
        19553114 1            [{'dbc': 'Arrival_MBMS.dbc', 'msg': 'BMS_Reque...   
        22174282 1                                                           []   

                              ecu_name_sa ecu_name_da cycle_time_range  \
CAN_Bus msg_id   is_extended                                             
1       1832     0                   None        None       [0.0, 0.5]   
        15739738 1                    DCU        None      [0.02, 0.1]   
        19422042 1                    DCU         BMS     [0.02, 0.02]   
        19553114 1                    DCU         BMS       [1.0, 1.0]   
        22174282 1                   COMM         DCU             None   

                                          SendOnChange dt_stat_mean_diff  \
CAN_Bus msg_id   is_extended                                               
1       1832     0            [None, None, None, None]         80.000682   
        15739738 1            [None, Yes, Yes, Yes, 1]        450.228606   
        19422042 1                  [None, None, None]         11.412454   
        19553114 1                  [None, None, None]          0.004060   
        22174282 1                                None               NaN   

                             dt_stat_mean_viol  dt_stat_cv_viol  
CAN_Bus msg_id   is_extended                                     
1       1832     0                        True            False  
        15739738 1                        True             True  
        19422042 1                        True             True  
        19553114 1                        None            False  
        22174282 1                        None             True  

[5 rows x 22 columns]

In [8]:
# # stats on dbc
utls.print_stat_log_msgs(mdf_df,dbc_db,file_export=file_export)

log msgs total number: 287 
log number of id wo dbc: 7
log number dublicated id: 205

dbc_db total number unique dbc 138
log number of used dbc: 67

dbc_db total number of messages in db: 3143
log number of used messages: 368

SA with ECU:
{None, 'INV', 'BMS', 'COMM', 'HMI', 'ARVLC', 'FBIO3', 'IDU', 'DCU'}

SA without ECU
{128.0, 0.0, 2.0, 136.0, 52.0, 58.0, 254.0}


messages with dt stat violation
number of messages with dT mean violation: 46
  (3, 234821722, 1): 2402.042331815789%: DriveCan.dbc::ModuleAlive; module_info.dbc::ModuleAlive
  (2, 234821768, 1): 2401.9925160256407%: DriveCan.dbc::ModuleAlive; module_info.dbc::ModuleAlive
  (1, 234821768, 1): 2401.971985564102%: DriveCan.dbc::ModuleAlive; module_info.dbc::ModuleAlive
  (3, 234821768, 1): 2401.956611769231%: DriveCan.dbc::ModuleAlive; module_info.dbc::ModuleAlive
  (3, 234821760, 1): 2401.9563593846156%: DriveCan.dbc::ModuleAlive; module_info.dbc::ModuleAlive
number of messages with CV violation: 137
  (1, 424888922, 1): 23

In [9]:
# prepare can matrix
(CAN_DBC_MAT,CAN_DBC_MSG_MAT,CAN_NODE_MAT,CAN_NODE_DA_MAT)=utls.get_can_com_matrix(mdf_df,file_export=file_export)
CAN_DBC_MAT.head();
CAN_DBC_MSG_MAT.head();
CAN_NODE_MAT.head();
CAN_NODE_DA_MAT.head();

CAN MATRIX reports written: C:\D\downloads\VI-15803\vcd_12_drop2_27_09_24.xlsx


,CAN1,CAN2,CAN3
ACU_Arrival.dbc,NaN,NaN,1.0
ADAS_J1939.dbc,1.0,1.0,1.0
ArrivalDTC.dbc,NaN,1.0,NaN
ArrivalSecurity.dbc,1.0,1.0,NaN
Arrival_Charging_Controller.dbc,1.0,1.0,NaN


CAN1  CAN2  CAN3
ACU_Arrival.dbc ACM_1   NaN   NaN   1.0
                ACM_2   NaN   NaN   1.0
                EDR_1   NaN   NaN   1.0
                EDR_2   NaN   NaN   1.0
ADAS_J1939.dbc  AC      1.0   1.0   1.0

,,CAN1,CAN2,CAN3
0.0,NA,NaN,NaN,1.0
2.0,NA,NaN,1.0,NaN
3.0,IDU,1.0,1.0,NaN
23.0,HMI,1.0,1.0,NaN
25.0,FBIO3,NaN,1.0,NaN


CAN1                                      CAN2        ...       \
           254.0 255.0 3.0  73.0 74.0 90.0 91.0  ALL 160.0 161.0  ... 23.0   
              NA    NA IDU ARVLC COMM  DCU  BMS   NA FBIO1 FBIO2  ...  HMI   
0.0  NA      NaN   NaN NaN   NaN  NaN  NaN  NaN  NaN   NaN   NaN  ...  NaN   
2.0  NA      NaN   NaN NaN   NaN  NaN  NaN  NaN  NaN   NaN   NaN  ...  NaN   
3.0  IDU     NaN   NaN NaN   NaN  NaN  NaN  NaN  1.0   NaN   NaN  ...  NaN   
23.0 HMI     NaN   NaN NaN   NaN  NaN  NaN  NaN  1.0   NaN   NaN  ...  NaN   
25.0 FBIO3   NaN   NaN NaN   NaN  NaN  NaN  NaN  NaN   NaN   NaN  ...  NaN   

                                             CAN3             
            25.0 254.0 255.0 59.0 90.0  ALL 254.0 255.0  ALL  
           FBIO3    NA    NA AVAS  DCU   NA    NA    NA   NA  
0.0  NA      NaN   NaN   NaN  NaN  NaN  NaN   NaN   NaN  1.0  
2.0  NA      NaN   1.0   NaN  NaN  NaN  NaN   NaN   NaN  NaN  
3.0  IDU     NaN   NaN   NaN  NaN  NaN  1.0   NaN   NaN  NaN  
23.0 HMI     NaN   NaN   NaN  NaN  NaN  1.0   NaN   NaN  NaN  
25.0 FBIO3   NaN   NaN   NaN  NaN  NaN  1.0   NaN   NaN  NaN  

[5 rows x 26 columns]

In [10]:
# # plot distributions

# from ipywidgets import interact, IntSlider

# # Sample data: Creating a list of DataFrames with random 'step_time' data
# # data_frames = [pd.DataFrame({'step_time': np.random.normal(loc=i*10, scale=5, size=100)}) for i in range(10)]
# CAN_LIST=list(mdf_d.keys())
# MSGS_LIST=[list(msg.keys()) for can, msg in mdf_df.items()]
# # Function to plot distributions with pagination

# def plot_distributions(can_id=0,page_num=0):
#     fig, axes = plt.subplots(2, 2, figsize=(10, 8))
#     fig.suptitle(f"{CAN_LIST[can_id]} Page {page_num + 1}")

#     # Flatten axes array for easier indexing
#     axes = axes.flatten()
#     data_frames=[df for msg, df in msgs1_df[CAN_LIST[can_id]].items()]
#     data_frames_titles=MSGS_LIST[can_id]
#     # Calculate the start and end index of the data frames to display on this page
#     start_idx = page_num * 4
#     end_idx = min(start_idx + 4, len(data_frames))

#     # Plot each data frame in a subplot
#     for i in range(4):
#         axes[i].clear()  # Clear previous plots
#         if start_idx + i < end_idx:
#             df = data_frames[start_idx + i]
#             axes[i].hist(df['dT'], bins=20, color='skyblue', edgecolor='black')
#             # axes[i].set_title(f'DataFrame {start_idx + i + 1}')
#             axes[i].set_title(f'{data_frames_titles[start_idx + i]}')
#         else:
#             axes[i].axis('off')  # Hide unused subplots

#     plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit suptitle
#     plt.show()




# # Number of pages required
# num_pages = (len(MSGS_LIST[0]) + 3) // 4

# can_widget = IntSlider(min=0, max=len(CAN_LIST)-1, step=1)
# page_widget = IntSlider(min=0, max=num_pages, step=1)
# msgs_widget = IntSlider(min=0, max=len(MSGS_LIST[0])-1, step=1, value=0)


# # 

    
# def update_pages_range(*args):
#     num_pages = (len(MSGS_LIST[can_widget.value]) + 3) // 4
#     page_widget.max=num_pages
    
# can_widget.observe(update_pages_range, 'value')
# interact(plot_distributions,can_id=can_widget, page_num=page_widget);